# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,21.02,74,43,6.73,PN,1722731261
1,1,cam pha mines,21.0167,107.3000,27.34,85,43,3.94,VN,1722731352
2,2,punta arenas,-53.1500,-70.9167,2.06,75,100,5.14,CL,1722731314
3,3,margaret river,-33.9500,115.0667,9.76,92,3,3.55,AU,1722731055
4,4,kavaratti,10.5669,72.6420,27.71,79,100,5.87,IN,1722731355


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity",
    color="City",
    alpha=0.5,
    scale=0.8,
    hover_cols=["City", "Lng", "Lat", "Max Temp", "Humidity"]
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[
    (city_data_df["Max Temp"] >= 27) & (city_data_df["Max Temp"] <= 35) & 
    (city_data_df["Wind Speed"] <= 5) & (city_data_df["Cloudiness"] < 50)
]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,cam pha mines,21.0167,107.3000,27.34,85,43,3.94,VN,1722731352
6,6,lincoln,40.8000,-96.6670,34.38,37,0,2.06,US,1722731137
15,15,oriximina,-1.7656,-55.8661,28.77,48,5,1.26,BR,1722731070
20,20,ishigaki,24.3448,124.1572,31.07,74,40,1.54,JP,1722731375
37,37,wailua homesteads,22.0669,-159.3780,27.44,73,40,4.63,US,1722731395


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,cam pha mines,21.0167,107.3000,27.34,85,43,3.94,VN,1722731352,
6,6,lincoln,40.8000,-96.6670,34.38,37,0,2.06,US,1722731137,
15,15,oriximina,-1.7656,-55.8661,28.77,48,5,1.26,BR,1722731070,
20,20,ishigaki,24.3448,124.1572,31.07,74,40,1.54,JP,1722731375,
37,37,wailua homesteads,22.0669,-159.3780,27.44,73,40,4.63,US,1722731395,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cam pha mines - nearest hotel: Hai Yen Luxury Hotel
lincoln - nearest hotel: New Victorian Suites Lincoln
oriximina - nearest hotel: No hotel found
ishigaki - nearest hotel: アートホテル石垣島
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
soto la marina - nearest hotel: Arellano
lihue - nearest hotel: Kauai Palms
recani - nearest hotel: Hotel Merzouga
tobelo - nearest hotel: JJ Guesthouse
griffith - nearest hotel: Quality Inn & Suites Hammond
la gi - nearest hotel: Lagi resort
xiahu - nearest hotel: No hotel found
terrace - nearest hotel: Sandman Hotel Terrace
santa marta - nearest hotel: Casa Mita
kapa'a - nearest hotel: Pono Kai Resort
kogota - nearest hotel: ビジネスホテルせんだい屋
utrik - nearest hotel: No hotel found
yutan - nearest hotel: No hotel found
el higo - nearest hotel: No hotel found
al wajh - nearest hotel: فندق رالين 2
pago pago - nearest hotel: No hotel found
shimoda - nearest hotel: ホテルウラガ
san andres - nearest hotel: Hotel Decameron Isleño
h

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,cam pha mines,21.0167,107.3000,27.34,85,43,3.94,VN,1722731352,Hai Yen Luxury Hotel
6,6,lincoln,40.8000,-96.6670,34.38,37,0,2.06,US,1722731137,New Victorian Suites Lincoln
15,15,oriximina,-1.7656,-55.8661,28.77,48,5,1.26,BR,1722731070,No hotel found
20,20,ishigaki,24.3448,124.1572,31.07,74,40,1.54,JP,1722731375,アートホテル石垣島
37,37,wailua homesteads,22.0669,-159.3780,27.44,73,40,4.63,US,1722731395,Hilton Garden Inn Kauai Wailua Bay
71,71,soto la marina,23.7667,-98.2167,27.03,80,28,3.88,MX,1722731437,Arellano
79,79,lihue,21.9789,-159.3672,27.77,75,40,4.63,US,1722731447,Kauai Palms
101,101,recani,31.2846,-4.2688,32.58,8,3,1.50,MA,1722731472,Hotel Merzouga
112,112,tobelo,1.7284,128.0095,27.82,78,19,1.18,ID,1722731485,JJ Guesthouse
118,118,griffith,41.5284,-87.4237,28.48,75,0,3.09,US,1722731493,Quality Inn & Suites Hammond


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity",
    color="Hotel Name",
    alpha=1,
    scale=0.8,
    hover_cols=["Hotel Name", "City", "Country", "Lng", "Lat", "Max Temp", "Humidity"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country,Max Temp)